In [3]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from numpy import genfromtxt
import numpy as np
import pandas as pd
import time
import random
import math

In [4]:
# load (downloaded if needed) the MNIST dataset

train = pd.read_csv('./datasets/train.csv')
test = pd.read_csv('./datasets/test.csv')

#y_train = np.array(df['label'].values)
#print(y_train)

X_train = (train.iloc[:,1:].values.astype('float32'))
y_train = (train.iloc[:,0].values.astype('int32'))
y_train = np.tile(y_train, (1, 1))
X_test = test.values.astype('float32')

In [5]:
print(X_train.shape[1])

784


In [31]:
class Neural_Network:
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.m = X_train.shape[0]
        self.input_layer_size = X_train.shape[1]
        self.hidden_layer_size = 25
        self.output_layer_size = 10
        self.epsilon = []
        self.theta1 = 0
        self.theta2 = 0
        self.J = 0
        self.grad = 0
        self.num_labels = 10
        
    def randInit(self, X, y):
        ils = self.input_layer_size
        hls = self.hidden_layer_size
        ols = self.output_layer_size
        
        L_in = ils
        L_out = hls
        
        self.epsilon.append(math.sqrt(6) / math.sqrt(L_in + L_out))
        
        L_in = hls
        L_out = ols
        
        self.epsilon.append(math.sqrt(6) / math.sqrt(L_in + L_out))
        
        #The theta values are of the form S(j + 1) x S(j) + 1 where S(j) i sthe size of the layer at j
        
        self.theta1 = np.dot(np.random.rand(hls, ils + 1), (2 * self.epsilon[0])) - self.epsilon[0]
        
        self.theta2 = np.dot(np.random.rand(ols, hls + 1), (2 * self.epsilon[1])) - self.epsilon[1]
        
    def sigmoid(self, z):
        return ( 1/(1 + np.exp(-z)))
    
    def sigmoidPrime(self, z):
        return np.multiply(z, (1-z))
    
    def costFunction(self):
        X = self.X_train
        y = self.y_train
        y = y.T
        
        #Foward propogation
        a1 = np.hstack((np.ones((self.m, 1)), X ))
        
        z2 = np.dot(a1, np.transpose(self.theta1))
        
        a2 = self.sigmoid(z2)
        a2 = np.hstack((np.ones((a2.shape[0], 1)), a2))
        
        z3 = np.dot(a2, np.transpose(self.theta2))
        
        a3 = self.sigmoid(z3)
        
        #Cost function
        
        numLabels_temp = np.tile([i for i in range(10)], (self.m, 1) )
        
        yMatrix_temp = np.tile(y, (1,self.num_labels) )
        
        yMatrix = np.equal(numLabels_temp, yMatrix_temp)
        
        cost_no_reg = np.sum(np.multiply(yMatrix, np.log(a3)) + np.multiply((1-yMatrix), np.log(1 - a3)))
        
        lam = 0
        
        reg = (lam / (2*self.m)) * ((np.sum(np.square(self.theta1[:, 2:None])) + np.sum(np.square(self.theta2[:, 2:None])))) ** 2
        
        J =  cost_no_reg + reg
        
        return J
        
    def backProp(self):
        X = self.X_train
        y = self.y_train
        y = y.T
        lam = 0
        
        numLabels_temp = np.tile([i for i in range(10)], (self.m, 1) )
        
        yMatrix_temp = np.tile(y, (1,self.num_labels) )
        
        yMatrix = np.equal(numLabels_temp, yMatrix_temp)
        
        delta2 = 0
        delta1 = 0
        
        for i in range(0, 5):
            
            a1 = np.append(1, X[i, :] )
            a1 = a1.reshape(1, a1.size)
        
            z2 = np.dot(a1, self.theta1.T)

            a2 = self.sigmoid(z2)
            a2 = np.append(1, a2)
            a2 = a2.reshape(1, a2.size)

            z3 = np.dot(a2, np.transpose(self.theta2))

            a3 = self.sigmoid(z3)
            a3 = a3.reshape(1, a3.size)
            
            yMatrix_i = yMatrix[i, :]
            yMatrix_i.reshape(1, yMatrix_i.size)
            
            delta3_i = np.subtract(a3, yMatrix_i)
            
            g2 = self.sigmoidPrime(np.insert(z2, 0, 1))
            g2 = g2.reshape(1, g2.size)
            
            delta2_i = np.multiply(np.dot(self.theta2.T, delta3_i.T), g2.T)
            delta2_i = delta2_i[1:, :].T
            
            delta2 = delta2 + np.dot(delta3_i.T, a2)
            
            delta1 = delta1 + np.dot(delta2_i.T, a1)
            
        theta1_no_bias = np.multiply((lam/self.m), self.theta1[:, 1:])
        theta2_no_bias = np.multiply((lam/self.m), self.theta2[:, 1:])
        
        theta1_bias = np.multiply((1/self.m), delta1[:, 1])
        theta1_bias = theta1_bias.reshape(theta1_bias.size, 1)
        
        theta2_bias = np.multiply((1/self.m), delta2[:, 1])
        theta2_bias = theta2_bias.reshape(theta2_bias.size, 1)
        
        Theta1_grad = np.concatenate((theta1_bias, np.add(np.multiply((1/self.m), delta1[:, 1:]), theta1_no_bias)), axis=1)
        Theta2_grad = np.concatenate((theta2_bias, np.add(np.multiply((1/self.m), delta2[:, 1:]), theta2_no_bias)), axis=1)

In [32]:
nn = Neural_Network(X_train, y_train)
nn.randInit(X_train, y_train)
nn.costFunction()
nn.backProp()